In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

EPOCHS = 300
BATCH_SIZE = 64
NUM_CLASSES = 10

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(
    y_train,
    num_classes=NUM_CLASSES,
    dtype='uint8'
)
y_test = tf.keras.utils.to_categorical(
    y_test,
    num_classes=NUM_CLASSES,
    dtype='uint8'
)
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [3]:
def data_gen(images, labels, is_training, batch_size=128):
    def preprocess_fn(image, label):
        x = tf.reshape(tf.cast(image, tf.float32), (32,32,3))
        x /= 255.0
        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_hue(x, 0.08)
        x = tf.image.random_saturation(x, 0.6, 1.6)
        x = tf.image.random_brightness(x, 0.05)
        x = tf.image.random_contrast(x, 0.7, 1.3)
        y = label
#         y = tf.one_hot(tf.cast(label, tf.uint8), 10)
        return x,y
    
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if is_training:
        dataset = dataset.shuffle(10000)
    
    dataset = dataset.apply(tf.data.experimental.map_and_batch(
        preprocess_fn, batch_size, num_parallel_batches=6, 
        drop_remainder=True if is_training else False))
    dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [4]:
train_ds = data_gen(x_train, y_train, is_training=True, batch_size=BATCH_SIZE)
val_ds = data_gen(x_test, y_test, is_training=False, batch_size=BATCH_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# iterator = train_ds.make_one_shot_iterator()
# x_sample, y_sample = iterator.get_next()

# import matplotlib.pyplot as plt
# import numpy as np

# np_sample = x_sample.numpy()
# print(np_sample[1].shape)
# plt.figure()
# plt.imshow(np_sample[50])
# plt.show()

# print(y_sample.shape)


In [6]:
from resnet34 import resnet34
from time import time
tensorboard = TensorBoard(log_dir="logs/cifar10_300_epoch_less_downsampled_resnet")

In [9]:
model = resnet34(input_shape=(32, 32, 3), initial_downsample=False,
                     num_classes=10)
model.summary()
optimizer = tf.keras.optimizers.Adam(lr = 0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 32, 32, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_33 (Batc (None, 32, 32, 64)   256         conv2d_33[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 32, 32, 64)   0           batch_normalization_v1_33[0][0]  
__________________________________________________________________________________________________
conv2d_34 

In [10]:
history = model.fit(
    train_ds.make_one_shot_iterator(), steps_per_epoch=len(x_train) // BATCH_SIZE,
    epochs=EPOCHS, validation_data=val_ds.make_one_shot_iterator(),
    validation_steps=len(x_test) // BATCH_SIZE,
    callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
781/781 [==============================] - 149s 190ms/step - loss: 1.7635 - acc: 0.3749 - val_loss: 1.4827 - val_acc: 0.4531
Epoch 2/300
781/781 [==============================] - 124s 159ms/step - loss: 1.2906 - acc: 0.5366 - val_loss: 1.3253 - val_acc: 0.5389
Epoch 3/300
781/781 [==============================] - 124s 159ms/step - loss: 1.0401 - acc: 0.6307 - val_loss: 1.3052 - val_acc: 0.5532
Epoch 4/300
781/781 [==============================] - 125s 160ms/step - loss: 0.8500 - acc: 0.7008 - val_loss: 0.9558 - val_acc: 0.6794
Epoch 5/300
781/781 [==============================] - 123s 158ms/step - loss: 0.7116 - acc: 0.7510 - val_loss: 0.8955 - val_acc: 0.6948
Epoch 6/300
781/781 [==============================] - 124s 158ms/step - loss: 0.6128 - acc: 0.7875 - val_loss: 0.8314 - val_acc: 0.7201
Epoch 7/300
781/781 [==============================] - 124s 158ms/step - loss: 0.5206 - acc: 0.8189 - val_loss: 0.7151 - val_acc: 

781/781 [==============================] - 124s 158ms/step - loss: 0.0283 - acc: 0.9903 - val_loss: 0.7379 - val_acc: 0.8556
Epoch 59/300
781/781 [==============================] - 121s 155ms/step - loss: 0.0254 - acc: 0.9913 - val_loss: 0.7708 - val_acc: 0.8548
Epoch 60/300
781/781 [==============================] - 124s 159ms/step - loss: 0.0241 - acc: 0.9915 - val_loss: 0.7361 - val_acc: 0.8554
Epoch 61/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0261 - acc: 0.9915 - val_loss: 1.0108 - val_acc: 0.8124
Epoch 62/300
781/781 [==============================] - 124s 159ms/step - loss: 0.0253 - acc: 0.9916 - val_loss: 0.7090 - val_acc: 0.8602
Epoch 63/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0263 - acc: 0.9912 - val_loss: 0.6935 - val_acc: 0.8589
Epoch 64/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0203 - acc: 0.9931 - val_loss: 0.8734 - val_acc: 0.8462
Epoch 65/300
781/781 [=========================

781/781 [==============================] - 123s 157ms/step - loss: 0.0126 - acc: 0.9957 - val_loss: 0.8030 - val_acc: 0.8572
Epoch 118/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0125 - acc: 0.9958 - val_loss: 0.7229 - val_acc: 0.8684
Epoch 119/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0114 - acc: 0.9961 - val_loss: 0.7365 - val_acc: 0.8642
Epoch 120/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0085 - acc: 0.9971 - val_loss: 0.8325 - val_acc: 0.8627
Epoch 121/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0128 - acc: 0.9959 - val_loss: 0.7133 - val_acc: 0.8713
Epoch 122/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0122 - acc: 0.9958 - val_loss: 0.8210 - val_acc: 0.8560
Epoch 123/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0138 - acc: 0.9954 - val_loss: 0.7862 - val_acc: 0.8666
Epoch 124/300
781/781 [==================

781/781 [==============================] - 123s 157ms/step - loss: 0.0068 - acc: 0.9977 - val_loss: 0.9041 - val_acc: 0.8554
Epoch 177/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0099 - acc: 0.9968 - val_loss: 0.6983 - val_acc: 0.8777
Epoch 178/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0052 - acc: 0.9983 - val_loss: 0.7439 - val_acc: 0.8754
Epoch 179/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0065 - acc: 0.9979 - val_loss: 0.7774 - val_acc: 0.8707
Epoch 180/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0075 - acc: 0.9975 - val_loss: 0.7946 - val_acc: 0.8702
Epoch 181/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0061 - acc: 0.9981 - val_loss: 0.7665 - val_acc: 0.8752
Epoch 182/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0071 - acc: 0.9975 - val_loss: 0.8402 - val_acc: 0.8685
Epoch 183/300
781/781 [==================

781/781 [==============================] - 123s 157ms/step - loss: 0.0089 - acc: 0.9972 - val_loss: 0.8033 - val_acc: 0.8718
Epoch 236/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0033 - acc: 0.9990 - val_loss: 0.7387 - val_acc: 0.8779
Epoch 237/300
781/781 [==============================] - 123s 158ms/step - loss: 0.0042 - acc: 0.9987 - val_loss: 0.7489 - val_acc: 0.8751
Epoch 238/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0053 - acc: 0.9981 - val_loss: 0.7713 - val_acc: 0.8791
Epoch 239/300
781/781 [==============================] - 123s 158ms/step - loss: 0.0074 - acc: 0.9977 - val_loss: 0.7632 - val_acc: 0.8744
Epoch 240/300
781/781 [==============================] - 123s 157ms/step - loss: 0.0048 - acc: 0.9985 - val_loss: 0.7404 - val_acc: 0.8762
Epoch 241/300
781/781 [==============================] - 122s 156ms/step - loss: 0.0042 - acc: 0.9987 - val_loss: 0.7486 - val_acc: 0.8726
Epoch 242/300
781/781 [==================

781/781 [==============================] - 125s 160ms/step - loss: 0.0036 - acc: 0.9988 - val_loss: 0.7911 - val_acc: 0.8782
Epoch 295/300
781/781 [==============================] - 127s 162ms/step - loss: 0.0030 - acc: 0.9990 - val_loss: 0.8003 - val_acc: 0.8826
Epoch 296/300
781/781 [==============================] - 124s 159ms/step - loss: 0.0047 - acc: 0.9984 - val_loss: 0.8457 - val_acc: 0.8712
Epoch 297/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0039 - acc: 0.9986 - val_loss: 0.8219 - val_acc: 0.8735
Epoch 298/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0040 - acc: 0.9985 - val_loss: 0.7998 - val_acc: 0.8765
Epoch 299/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0038 - acc: 0.9986 - val_loss: 0.7667 - val_acc: 0.8839
Epoch 300/300
781/781 [==============================] - 125s 160ms/step - loss: 0.0044 - acc: 0.9987 - val_loss: 0.7681 - val_acc: 0.8770


In [1]:
model.save('cifar10_300_epoch_run', overwrite=True)

NameError: name 'model' is not defined